# Import Packages

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import pickle

# Training data Import

In [2]:
data = pd.read_csv('Dataset\\Train\\train_data.csv')
X_train = data.iloc[:, :49].values
y_train = data.iloc[:, -1].values
X_train = X_train.reshape((X_train.shape[0], 7, 7, 1))

# Import Submodels

In [3]:
submodel_path = "Models\\SubModel"
submodels_filenames = os.listdir(submodel_path)

In [4]:
submodels = []
for filename in submodels_filenames:
    model = tf.keras.models.load_model("Models\\SubModel\\"+filename)
    model.trainable = False
    submodels.append(model)

# Predictions from submodels

In [5]:
base_model_predictions = []
for model in submodels:
    base_model_predictions.append(model.predict([X_train,X_train,X_train,X_train,X_train]))

# Generate predictions using weighted averaging
base_model_accuracy = []
for preds in base_model_predictions:
    accuracy = accuracy_score(y_train, preds.argmax(axis=1))
    base_model_accuracy.append(accuracy)

4219/4219 [==============================] - 26s 6ms/step


In [8]:
X_train_stacked = np.concatenate(base_model_predictions, axis=-1)

# Stacking LSTM

In [8]:
def lr_schedule(epoch):
    initial_lr = 0.01 
    drop = 0.9  
    epochs_drop = 20
    lr = initial_lr * np.power(drop, np.floor((1 + epoch) / epochs_drop))
    return lr

In [9]:
stacker_lstm_model = models.Sequential([
layers.Reshape((-1, len(submodels) * 18), input_shape=(len(submodels) * 18,)),  # Reshape to add the third dimension
layers.LSTM(64, activation='tanh'),
layers.Dense(18, activation='softmax')
])
stacker_lstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(lr_schedule)
stacker_lstm_model.fit(X_train_stacked, y_train, epochs=200, batch_size=32, verbose=1, callbacks=[lr_scheduler])

Epoch 1/200


4219/4219 [==============================] - 17s 4ms/step - loss: 0.5902 - accuracy: 0.7613 - lr: 0.0100
Epoch 2/200
4219/4219 [==============================] - 15s 4ms/step - loss: 0.5570 - accuracy: 0.7714 - lr: 0.0100
Epoch 3/200
4219/4219 [==============================] - 22s 5ms/step - loss: 0.5467 - accuracy: 0.7734 - lr: 0.0100
Epoch 4/200
4219/4219 [==============================] - 21s 5ms/step - loss: 0.5407 - accuracy: 0.7759 - lr: 0.0100
Epoch 5/200
4219/4219 [==============================] - 19s 4ms/step - loss: 0.5372 - accuracy: 0.7772 - lr: 0.0100
Epoch 6/200
4219/4219 [==============================] - 10s 2ms/step - loss: 0.5326 - accuracy: 0.7795 - lr: 0.0100
Epoch 7/200
4219/4219 [==============================] - 10s 2ms/step - loss: 0.5296 - accuracy: 0.7798 - lr: 0.0100
Epoch 8/200
4219/4219 [==============================] - 9s 2ms/step - loss: 0.5273 - accuracy: 0.7815 - lr: 0.0100
Epoch 9/200
4219/4219 [==============================] - 9s 2ms

# Stacking CNN

In [11]:
def lr_schedule_cnn(epoch):
    initial_lr = 0.01 
    drop = 0.5  
    epochs_drop = 50
    lr = initial_lr * np.power(drop, np.floor((1 + epoch) / epochs_drop))
    return lr

In [12]:
stacker_cnn_model = models.Sequential([
    layers.Reshape((-1, len(submodels) * 18), input_shape=(len(submodels) * 18,)),  # Reshape to add the third dimension
    layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'),
    layers.GlobalMaxPooling1D(),  # Global max pooling to reduce dimensionality
    layers.Dense(18, activation='softmax')
])

X_train_stacked = np.concatenate(base_model_predictions, axis=-1)
stacker_cnn_model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(lr_schedule_cnn)
stacker_cnn_model.fit(X_train_stacked, y_train, epochs=200, batch_size=32, verbose=1, callbacks=[lr_scheduler])

Epoch 1/200
4219/4219 [==============================] - 9s 2ms/step - loss: 0.5911 - accuracy: 0.7625 - lr: 0.0100
Epoch 2/200
4219/4219 [==============================] - 7s 2ms/step - loss: 0.5628 - accuracy: 0.7700 - lr: 0.0100
Epoch 3/200
4219/4219 [==============================] - 7s 2ms/step - loss: 0.5566 - accuracy: 0.7723 - lr: 0.0100
Epoch 4/200
4219/4219 [==============================] - 8s 2ms/step - loss: 0.5530 - accuracy: 0.7726 - lr: 0.0100
Epoch 5/200
4219/4219 [==============================] - 7s 2ms/step - loss: 0.5493 - accuracy: 0.7743 - lr: 0.0100
Epoch 6/200
4219/4219 [==============================] - 7s 2ms/step - loss: 0.5473 - accuracy: 0.7766 - lr: 0.0100
Epoch 7/200
4219/4219 [==============================] - 7s 2ms/step - loss: 0.5465 - accuracy: 0.7757 - lr: 0.0100
Epoch 8/200
4219/4219 [==============================] - 8s 2ms/step - loss: 0.5447 - accuracy: 0.7770 - lr: 0.0100
Epoch 9/200
4219/4219 [==============================] - 8s 2ms/step - l

# Combined Model LSTM

In [47]:
def lr_schedule_combined(epoch):
    initial_lr = 0.01 
    drop = 0.9
    epochs_drop = 10
    lr = initial_lr * np.power(drop, np.floor((1 + epoch) / epochs_drop))
    return lr

In [48]:
stacker_lstm_output = stacker_lstm_model.predict(X_train_stacked)
stacker_cnn_output = stacker_cnn_model.predict(X_train_stacked)
X_combined = np.concatenate([stacker_lstm_output, stacker_cnn_output], axis=1)

# Reshape the combined output to add a time dimension
X_combined = np.reshape(X_combined, (X_combined.shape[0], 1, X_combined.shape[1]))

# Build another model on top of the concatenated outputs
combined_model = models.Sequential([
    #layers.Concatenate(),
    layers.LSTM(64, activation='tanh'),
    layers.Dense(18, activation='softmax')
])

combined_model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(lr_schedule_combined)
combined_model.fit(X_combined, y_train, epochs=200, batch_size=32, verbose=1, callbacks=[lr_scheduler])

4219/4219 [==============================] - 6s 1ms/step
Epoch 1/200
4219/4219 [==============================] - 19s 4ms/step - loss: 0.5107 - accuracy: 0.7928 - lr: 0.0100
Epoch 2/200
4219/4219 [==============================] - 15s 3ms/step - loss: 0.4832 - accuracy: 0.7994 - lr: 0.0100
Epoch 3/200
4219/4219 [==============================] - 15s 4ms/step - loss: 0.4781 - accuracy: 0.8003 - lr: 0.0100
Epoch 4/200
4219/4219 [==============================] - 15s 4ms/step - loss: 0.4756 - accuracy: 0.8006 - lr: 0.0100
Epoch 5/200
4219/4219 [==============================] - 15s 4ms/step - loss: 0.4733 - accuracy: 0.8016 - lr: 0.0100
Epoch 6/200
4219/4219 [==============================] - 15s 4ms/step - loss: 0.4718 - accuracy: 0.8016 - lr: 0.0100
Epoch 7/200
4219/4219 [==============================] - 15s 3ms/step - loss: 0.4704 - accuracy: 0.8024 - lr: 0.0100
Epoch 8/200
4219/4219 [==============================] - 15s 3ms/step - loss: 0.4698 - accuracy: 0.8026 - lr: 0.0100
Epoch 9

# Save Models

In [80]:
import pickle
with open(f'Models\\GlobalModel\\Global_Model.pkl', 'wb') as f:
    pickle.dump({
        'base_models': submodels,
        'stacker_lstm_model': stacker_lstm_model,
        'stacker_cnn_model': stacker_cnn_model,
        'combined_model': combined_model
    }, f)
